In [28]:
import pickle
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [192]:
with open('pickle_files/story_and_topic.pickle','rb') as read_file:
    df= pickle.load(read_file)

In [178]:
df.head(5)

,title,author,region,country,continent_name,body,topic_name,clean_text
0,A Clever Thief,Hindu Tales from the Sanskrit,Indian,India,Asia,"Chapter I A certain man, named Hari-Sarman, wh...",Poor vs. Rich,chapter i a certain man name hari sarman w...
1,A Lac of Rupees for a Piece of Advice,Joseph Jacobs,Indian,India,Asia,A poor blind Brahman and his wife were depende...,Poor vs. Rich,a poor blind brahman and wife be dependent on...
2,A Leaf from the Sky,Hans Christian Andersen,Danish Nordic Scandinavian,Scandinavia,Europe,"High up yonder, in the thin clear air, flew an...",Animal Characters,high up yonder in the thin clear air fly a...
3,A Legend of Confucius,The Chinese Fairy Book,Chinese,China,Asia,"When Confucius came to the earth, the Kilin, t...",Love,when confucius come to the earth the kilin ...
4,A Lesson for Kings,Joseph Jacobs,Indian,India,Asia,"Once upon a time, when Brahma-datta was reigni...",Holiday,once upon a time when brahma datta be reign ...


# Create word count dataframe with topics&continents

In [179]:
def get_doc_term(vectorizer, doc):
    vec = vectorizer.fit(doc)
    matrix = vec.transform(doc)
    term = vec.get_feature_names()
    return vec, matrix, term

In [180]:
cv, cv_matrix, cv_terms = get_doc_term(CountVectorizer(stop_words = 'english', min_df = 3), df.clean_text)

In [181]:
word_doc = pd.DataFrame(cv_matrix.toarray(), columns = cv.get_feature_names())

In [203]:
word_topic_continent = pd.concat([word_doc,df.topic_name, df.continent_name], axis = 1)
word_topic_continent.head(5)

,aard,aback,abandon,abase,abash,abate,abated,abbess,abbey,abbot,...,zobeida,zodiac,zoological,ægean,ægir,æsir,æson,æsop,topic_name,continent_name
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Poor vs. Rich,Asia
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Poor vs. Rich,Asia
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Animal Characters,Europe
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Love,Asia
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Holiday,Asia


# Word count in each topic

In [219]:
by_topic = word_topic_continent.groupby('topic_name').sum().transpose()
by_topic.columns

Index(['Animal Characters', 'Family', 'Farm', 'Holiday', 'Humans & Animals',
       'Love', 'Nature', 'Poor vs. Rich'],
      dtype='object', name='topic_name')

# Word count in each continent

In [220]:
by_continent = word_topic_continent.groupby('continent_name').sum().transpose()
by_continent.columns

Index(['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America',
       'Unknown'],
      dtype='object', name='continent_name')

# Save in excel for Tableau Word Clouds

In [251]:
Africa = by_continent['Africa'].sort_values(ascending = False)[0:200]
Asia = by_continent['Asia'].sort_values(ascending = False)[0:200]
Europe = by_continent['Europe'].sort_values(ascending = False)[0:200]
NorthAmerica = by_continent['North America'].sort_values(ascending = False)[0:200]
Oceania = by_continent['Oceania'].sort_values(ascending = False)[0:200]
SouthAmerica = by_continent['South America'].sort_values(ascending = False)[0:200]

In [252]:
animalcharacters = by_topic['Animal Characters'].sort_values(ascending = False)[0:200]
family = by_topic['Family'].sort_values(ascending = False)[0:200]
holiday = by_topic['Holiday'].sort_values(ascending = False)[0:200]
humananimals = by_topic['Humans & Animals'].sort_values(ascending = False)[0:200]
love = by_topic['Love'].sort_values(ascending = False)[0:200]
nature = by_topic['Nature'].sort_values(ascending = False)[0:200]
poorvsrich = by_topic['Poor vs. Rich'].sort_values(ascending = False)[0:200]
farm = by_topic['Farm'].sort_values(ascending = False)[0:200]

# Calculate percentage of topics in each continent and save in excel

In [198]:
topic_by_continent = word_topic_continent.groupby(['continent_name','topic_name']).agg({'topic_name':'count'}).groupby(level=0).apply(
     lambda x:  100*x / x.sum()).unstack()

In [279]:
topic_by_continent = pd.DataFrame(topic_by_continent).round(2)

In [257]:
# topic_by_continent.to_excel('topic_by_continent.xlsx')

In [276]:
topic_by_continent2 = word_topic_continent.groupby(['continent_name','topic_name']).agg({'topic_name':'count'}).groupby(level=0).apply(
     lambda x:  100*x / x.sum()).round(2)

In [277]:
# topic_by_continent2.to_excel('topic_by_continent2.xlsx')